# FastAPI 子应用

参考：[sub-applications](https://fastapi.tiangolo.com/zh/advanced/sub-applications/)

如果你需要有两个独立的 FastAPI 应用程序，它们有自己独立的 OpenAPI 和自己的文档 UI，你可以分别拥有主应用程序和“挂载”的一个或多个子应用程序。

## 挂载 FastAPI 应用程序

“挂载”意味着在特定路径中添加完全“独立”的应用程序，然后处理该路径下的所有内容，并在该子应用程序中声明路径操作。

### 顶层 FastAPI 应用程序

首先，创建主要的顶层 FastAPI 应用程序及其路径操作：

In [1]:
!pip install --upgrade -q jinja2 flask fastapi

In [2]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/app")
def read_main():
    return {"message": "Hello World from main app"}

### 子应用

然后，创建子应用程序及其路径操作。

这里的子应用程序只是另一个标准的 FastAPI 应用程序，但它将被“挂载”：

In [3]:
subapi = FastAPI()


@subapi.get("/sub")
def read_sub():
    return {"message": "Hello World from sub API"}

### 挂载子应用程序

在顶层应用程序 `app` 中，挂载子应用程序 `subapi`。

在这种情况下，它将被挂载在 `/subapi` 路径下：

In [4]:
app.mount("/subapi", subapi)

## 挂载其他 WSGI 组件

可以使用 `WSGIMiddleware` 来包装你的 WSGI 应用，如：Flask，Django 等。

使用中间件包装 WSGI 应用（例如 Flask），之后将其挂载到某一个路径下：

In [5]:
from fastapi import FastAPI
from fastapi.middleware.wsgi import WSGIMiddleware
from flask import Flask, request

flask_app = Flask(__name__)

@flask_app.route("/")
def flask_main():
    name = request.args.get("name", "World")
    return f"Hello, {name} from Flask!"

@app.get("/v2")
def read_main():
    return {"message": "Hello World"}


app.mount("/v1", WSGIMiddleware(flask_app))

```{note}
现在，所有定义在 `/v1/` 路径下的请求将会被 Flask 应用处理。其余的请求则会被 FastAPI 处理。
```

In [ ]:
if __name__ == "__main__":
    import asyncio
    import uvicorn
    config = uvicorn.Config(app, port=3002, reload=False,)
    server = uvicorn.Server(config)
    loop = asyncio.get_event_loop()
    loop.create_task(server.serve())

INFO:     Started server process [15284]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:3002 (Press CTRL+C to quit)


INFO:     127.0.0.1:61263 - "GET / HTTP/1.1" 404 Not Found
